In [98]:
from pathlib import Path
import os,sys
import pickle
import pandas as pd
import importlib
from tqdm.notebook import tqdm

sys.path.insert(0, str(Path().resolve().parents[1]))
import fusemix.pipeline as pipeline 
from fusemix.imputation import MultipleImputer

from ucimlrepo import fetch_ucirepo
from gower import gower_matrix

import fusemix.clustering as clust_utils 

import fusemix.mige as migeClust

importlib.reload(clust_utils)
from fusemix.fcmeans import FCM

from sklearn.impute import KNNImputer
import warnings

import importlib
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans, SpectralClustering
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

import seaborn as sns 

warnings.filterwarnings("ignore")

In [ ]:
migeClust.mige(test_data,2,cat_mask=test_data_complete['cat_mask'])

NameError: name 'gower_matrix' is not defined

In [96]:
test_data[1]

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_elbow_involvement,scalp_involvement,...,focal_hypergranulosis,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_the_basal_layer,spongiosis,saw_tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_infiltrate,band_like_infiltrate,age
0,2.000000,2.000000,0.0,3.0,0.000000,0.000000e+00,0.000000e+00,0.0,1.0,0.0,...,0.000000e+00,0.0,0.000000e+00,3.0,0.0,0.000000e+00,0.000000e+00,1.000000,0.000000,55.000000
1,2.520833,2.379717,3.0,2.0,0.888551,0.000000e+00,4.166667e-02,0.0,1.0,1.0,...,0.000000e+00,0.0,2.314744e-09,0.0,0.0,-3.261499e-09,0.000000e+00,2.016956,0.000000,45.482626
2,2.000000,1.000000,2.0,3.0,1.175595,2.333333e+00,9.123160e-10,3.0,0.0,0.0,...,2.000000e+00,0.0,2.458333e+00,3.0,2.0,2.083333e-02,0.000000e+00,2.519150,3.000000,26.000000
3,2.000000,2.000000,2.0,0.0,0.138350,-1.047109e-08,4.166667e-02,0.0,3.0,2.0,...,0.000000e+00,3.0,2.314744e-09,0.0,0.0,-3.261499e-09,0.000000e+00,1.548547,0.000000,40.000000
4,2.000000,3.000000,2.0,2.0,2.000000,2.000000e+00,0.000000e+00,2.0,0.0,0.0,...,2.000000e+00,2.0,3.000000e+00,2.0,3.0,0.000000e+00,0.000000e+00,2.000000,3.000000,45.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,2.000000,1.689778,1.0,0.0,1.000000,-1.047109e-08,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,0.0,0.000000e+00,1.0,0.0,0.000000e+00,0.000000e+00,1.406412,0.000000,25.000000
354,1.804511,1.736915,1.0,0.0,1.000000,0.000000e+00,4.583333e-01,0.0,0.0,0.0,...,-6.549961e-09,1.0,2.314744e-09,1.0,0.0,-3.261499e-09,-2.292486e-09,2.000000,0.000000,36.000000
355,2.577920,2.000000,2.0,2.0,3.000000,2.270833e+00,0.000000e+00,2.0,0.0,0.0,...,1.708333e+00,0.0,2.437500e+00,0.0,3.0,0.000000e+00,-2.292486e-09,2.000000,2.145833,45.630488
356,2.000000,1.605899,3.0,1.0,2.000000,2.083333e+00,0.000000e+00,2.0,0.0,0.0,...,3.000000e+00,0.0,2.000000e+00,0.0,1.0,0.000000e+00,0.000000e+00,2.218750,3.000000,50.000000


In [8]:
def read_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def write_pickle(var, path):
    with open(path, 'wb') as f:
       pickle.dump(var, f)

In [32]:
test_data_complete = read_pickle("../../test_data/fetched/dataset_33.pkl")
test_data_missing = read_pickle("../../test_data/missing_data/33/0.75_0.5_0.0/data_pipeline_0.pkl")
test_data = read_pickle("../../test_data/imputed_data/33/0.75_0.5_0.0/data_imputed_0.pkl")

In [ ]:
test_data_missing.amputer.incomplete_dataset.head(2)

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_elbow_involvement,scalp_involvement,...,focal_hypergranulosis,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_the_basal_layer,spongiosis,saw_tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_infiltrate,band_like_infiltrate,age
0,2.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,55.0
1,NaN,NaN,3.0,2.0,NaN,0.0,NaN,0.0,1.0,1.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN


In [55]:
from rpy2.ipython import html
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri, NULL

MixtureMissing = importr('MixtureMissing')


In [43]:
from rpy2.robjects.conversion import localconverter

r('library(MixtureMissing)')  

mcnm = r["MCNM"] 

df_py = pd.DataFrame(test_data_missing.amputer.incomplete_dataset)

with localconverter(pandas2ri.converter):
    df_r = pandas2ri.py2rpy(df_py)

res_r = mcnm(
    X=df_r,
    G=r['as.numeric'](2),
    epsilon = 1
)



Mixture: Contaminated Normal (CN)
Data Set: Incomplete
Initialization: kmedoids 

Fitting G = 2 was successful with 3/20 iterations

The fitted mixture model with G = 2 has BIC = 19652.56



In [74]:
def run_mcnm(df_py, G=2, seed=1,init_method="kmeans", epsilon_start=1e-3, max_iter=20, max_epsilon=1.0):
    """
    Run MCNM on a pandas DataFrame with adaptive epsilon until convergence.
    
    Parameters:
        df_py: pandas.DataFrame
        G: int, number of clusters
        init_method: str, initialization method
        epsilon_start: float, starting epsilon
        max_iter: int, maximum iterations per call
        max_epsilon: float, maximum epsilon to try
    
    Returns:
        res_py: Python dict of the MCNM result
    """
    eps = epsilon_start
    res_r = NULL
    r['set.seed'](seed)

    with localconverter(pandas2ri.converter):
        df_r = pandas2ri.py2rpy(df_py)
    
    mcnm = r["MCNM"]
    
    while res_r == NULL and eps <= max_epsilon:
        try:
            res_r = mcnm(
                X=df_r,
                G=r['as.integer'](G),
                init_method=init_method,
                epsilon=r['as.numeric'](eps),
                max_iter=r['as.integer'](max_iter),
                progress=False
            )
            
        except Exception as e:
            res_r = None
        
        if res_r == NULL:
            eps *= 10
    
    if res_r == NULL:
        raise RuntimeError("MCNM did not converge even after increasing epsilon.")
    
    clusters_r = res_r.rx2('clusters')
    with localconverter(pandas2ri.converter):
        clusters_py = pd.Series(pandas2ri.rpy2py(clusters_r))
    
    return np.array(clusters_py)

In [75]:
result = run_mcnm(df_py)

In [76]:
result

array([1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1,
       2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2,
       2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2,
       1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2,
       1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2,
       2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2,
       2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1,
       2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1,

In [77]:
fcm = FCM(n_clusters=2)
fcm.fit(test_data[0].to_numpy())

In [23]:
labels = fcm.predict(test_data[0].to_numpy())

In [42]:
X = X.loc[:, ~X.columns.duplicated()]
print(X.shape)

(366, 34)


In [43]:
dataset.variables = dataset.variables.loc[~dataset.variables['name'].duplicated(),:]
dataset.variables = dataset.variables.reset_index(drop=True)

# sanitize names
X.columns = X.columns.str.replace(r'[^0-9a-zA-Z_]+', '_', regex=True)
dataset.variables['name'] = dataset.variables['name'].str.replace(r'[^0-9a-zA-Z_]+', '_', regex=True)

In [46]:
cat_variables = dataset.variables.loc[(dataset.variables['type'] == 'Categorical') | (dataset.variables['type'] == 'Binary'), 'name']
cat_variables

10    family_history
Name: name, dtype: object

In [47]:
features = dataset.data.features.columns

In [49]:
features.str.replace(r'[^0-9a-zA-Z_]+', '_', regex=True)

Index(['erythema', 'scaling', 'definite_borders', 'itching',
       'koebner_phenomenon', 'polygonal_papules', 'follicular_papules',
       'oral_mucosal_involvement', 'knee_elbow_involvement',
       'scalp_involvement', 'family_history', 'melanin_incontinence',
       'eosinophils_in_the_infiltrate', 'pnl_infiltrate',
       'fibrosis_of_the_papillary_dermis', 'exocytosis', 'acanthosis',
       'hyperkeratosis', 'parakeratosis', 'clubbing_of_the_rete_ridges',
       'elongation_of_the_rete_ridges',
       'thinning_of_the_suprapapillary_epidermis', 'spongiform_pustule',
       'munro_microabcess', 'focal_hypergranulosis',
       'disappearance_of_the_granular_layer',
       'vacuolisation_and_damage_of_the_basal_layer', 'spongiosis',
       'saw_tooth_appearance_of_retes', 'follicular_horn_plug',
       'perifollicular_parakeratosis', 'inflammatory_monoluclear_infiltrate',
       'band_like_infiltrate', 'age'],
      dtype='object')

In [5]:
test_data_complete['X_complete'].head(2)

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
0,5.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0
1,5.0,4.0,4.0,5.0,7.0,10.0,3.0,2.0,1.0


In [6]:
test_data[0].head(2)

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
0,5.0,1.0,1.000000,1.0,2.0,1.0,3.000000,1.0,1.000000
1,5.0,4.0,5.756944,5.0,7.0,10.0,5.985938,2.0,1.889583


In [7]:
test_data_complete['num_classes']

2

In [8]:
test_data_missing.amputer.incomplete_dataset

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
0,5.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0
1,5.0,4.0,NaN,5.0,7.0,10.0,NaN,2.0,NaN
2,3.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0
3,6.0,8.0,8.0,1.0,3.0,4.0,NaN,7.0,1.0
4,4.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...
694,3.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0
695,2.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0
696,5.0,10.0,10.0,3.0,7.0,3.0,8.0,10.0,2.0
697,4.0,8.0,NaN,4.0,3.0,4.0,NaN,6.0,NaN


In [79]:
# this needs to be set for each experiment, but not for each sampling
seed = 42
rng = np.random.default_rng(seed)
# ================================= #

cat_mask = test_data_complete['cat_mask']
view = test_data[0]
n_features = view.shape[1]
p_min = 0.75*n_features
p_max = 0.85*n_features

num_projections = 3

In [80]:
def generate_projection(data,cat_mask):
    # create projection of the view
    view = data.copy()
    alfa = rng.random()
    selected_features = rng.choice(a=range(n_features), size=round(p_min+alfa*(p_max-p_min)), replace=False)  
    cat_mask_projected = cat_mask[selected_features]
    projected_view = view.iloc[:,selected_features]
    return (projected_view,cat_mask_projected)

all_data  = []
for view in test_data:
    for i in range(num_projections):
        all_data.append(generate_projection(data=view, cat_mask=cat_mask))

In [81]:
def compute_sparse_similarity(data, cat_mask, k):    
    gower_dist = gower_matrix(data, cat_features=cat_mask)
    neighbors_idx = np.argsort(gower_dist, axis=1)[:, 1:k+1]

    # build sparse matrix 
    rows = np.repeat(np.arange(gower_dist.shape[0]), k)
    cols = neighbors_idx.flatten()
    A = csr_matrix((gower_dist[rows, cols], (rows, cols)), shape=gower_dist.shape)

    # simmetrize matrix  
    A = 0.5 * (A + A.T)
    # distance -> similarity
    A.data = 1-A.data

    # normalize where each row sums to 1
    row_sums = np.array(A.sum(axis=1)).flatten()
    row_sums[row_sums == 0] = 1  # avoid division by zero
    A = A.multiply(1 / row_sums[:, None])
    return A

all_projections = [compute_sparse_similarity(prj_view,prj_mask,10) for (prj_view, prj_mask) in all_data]

In [82]:
def compute_spectral(sparse_affinity_mat):
    sc = SpectralClustering(n_clusters=test_data_complete['num_classes'],
                            random_state=seed,
                            affinity="precomputed"
                            #assign_labels="cluster_qr"
                            )

    return sc.fit(sparse_affinity_mat).labels_

all_labels = [compute_spectral(aff_mat) for aff_mat in all_projections]

In [83]:
CO = (np.array(all_labels)[:, :, None] == np.array(all_labels)[:, None, :]).mean(axis=0)

In [84]:
def consensus_clustering(CO, threshold=0.5):
    """
    labels: (M, N) array of cluster assignments
    threshold: co-association cutoff (e.g., 0.5)
    """
    # threshold
    adj = (CO >= threshold).astype(int)
    graph = csr_matrix(adj)

    sc = SpectralClustering(n_clusters=test_data_complete['num_classes'],
                            random_state=seed,
                            affinity="precomputed"
                            #assign_labels="cluster_qr"
                            )
    return sc.fit(graph).labels_

In [85]:
predicted_labels = consensus_clustering(CO)
true_labels = test_data_complete['y_complete'].to_numpy().ravel()

In [86]:
num_clusters = test_data_complete['num_classes']

In [87]:
feature_names = test_data_missing.amputer.incomplete_dataset.columns.to_numpy()



In [88]:
test_data_complete['X_complete']

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_elbow_involvement,scalp_involvement,...,focal_hypergranulosis,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_the_basal_layer,spongiosis,saw_tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_infiltrate,band_like_infiltrate,age
0,2.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,55.0
1,3.0,3.0,3.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0
2,2.0,1.0,2.0,3.0,1.0,3.0,0.0,3.0,0.0,0.0,...,2.0,0.0,2.0,3.0,2.0,0.0,0.0,2.0,3.0,26.0
3,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,40.0
4,2.0,3.0,2.0,2.0,2.0,2.0,0.0,2.0,0.0,0.0,...,2.0,2.0,3.0,2.0,3.0,0.0,0.0,2.0,3.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,25.0
362,3.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,36.0
363,3.0,2.0,2.0,2.0,3.0,2.0,0.0,2.0,0.0,0.0,...,3.0,0.0,3.0,0.0,3.0,0.0,0.0,2.0,3.0,28.0
364,2.0,1.0,3.0,1.0,2.0,3.0,0.0,2.0,0.0,0.0,...,3.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,3.0,50.0


In [19]:
# k_cmm = kcmm.KCMM(categorical=feature_names[cat_mask], n_clusters=num_clusters)
# k_cmm_labels = k_cmm.fit(test_data_missing.amputer.incomplete_dataset.to_numpy()).labels_






In [89]:

predicted_mica = clust_utils.compute_MICA(test_data,num_clusters,seed)
predicted_kpod =  clust_utils.compute_kpod(test_data_missing.amputer.incomplete_dataset,num_clusters,seed)
predicted_fusemix = clust_utils.compute_fusemix(test_data,cat_mask,
                    num_clusters,
                    nn_snf=10,
                    seed=seed)
predicted_spectral_single = clust_utils.compute_spectral(test_data[0],cat_mask,num_clusters,seed)
predicted_kmeans_single = clust_utils.compute_kmeans(test_data[0],num_clusters=num_clusters,seed=seed)

In [91]:
print(clust_utils.external_metrics(true_labels,predicted_labels))
print(clust_utils.external_metrics(true_labels,predicted_mica))
print(clust_utils.external_metrics(true_labels,predicted_kpod[0]))
print(clust_utils.external_metrics(true_labels,predicted_spectral_single))
print(clust_utils.external_metrics(true_labels,predicted_kmeans_single))
print(clust_utils.external_metrics(true_labels,result))

{'ari': 0.8899934768667916, 'ami': 0.8963729008562555, 'vm': 0.8986425869864828, 'cs': 0.9004576041625691}
{'ari': 0.17724996961614548, 'ami': 0.26224174370192505, 'vm': 0.2784031857246782, 'cs': 0.27963975994231954}
{'ari': 0.024244703045606764, 'ami': 0.014809012187111446, 'vm': 0.03956889971534963, 'cs': 0.044933894632049554}
{'ari': 0.6923172813608134, 'ami': 0.8413054179736597, 'vm': 0.844756575205108, 'cs': 0.8425098252110841}
{'ari': 0.16433537268071707, 'ami': 0.2456967097560746, 'vm': 0.261916639687908, 'cs': 0.25947859497284254}
{'ari': 0.1215632424306918, 'ami': 0.22425595002105814, 'vm': 0.22895206841946208, 'cs': 0.4072055026509423}


In [ ]:
sparse_mat = compute_sparse_similarity(test_data_complete['X_complete'], cat_mask, 10)
cca_spectral = SpectralClustering(n_clusters=num_clusters, random_state=seed, affinity="precomputed").fit(sparse_mat).labels_

In [ ]:
print(external_metrics(cca_spectral,predicted_labels))
print(external_metrics(cca_spectral,predicted_mica))
print(external_metrics(cca_spectral,predicted_kpod[0]))

{'ari': 0.9533413030013026, 'ami': 0.9035578701309003, 'vm': 0.9036663413245779, 'cs': 0.9024901520325989}
{'ari': 0.9475212988640727, 'ami': 0.8996134776455079, 'vm': 0.8997270840673357, 'cs': 0.9040818421466608}
{'ari': 0.9075765567571501, 'ami': 0.8458290954661353, 'vm': 0.8460044858896876, 'cs': 0.8545869823864344}


In [ ]:
cca_kmeans = KMeans(n_clusters=num_clusters).fit(test_data_complete['X_complete']).labels_

In [ ]:
print(external_metrics(cca_kmeans,predicted_labels))
print(external_metrics(cca_kmeans,predicted_mica))
print(external_metrics(cca_kmeans,predicted_kpod[0]))

{'ari': 0.9076090768258467, 'ami': 0.8543718396534232, 'vm': 0.8545372912352716, 'cs': 0.84495049853657}
{'ari': 0.9589662318964268, 'ami': 0.9227555595967384, 'vm': 0.9228438676542211, 'cs': 0.9180458501538769}
{'ari': 0.9531089026797084, 'ami': 0.9011721824165765, 'vm': 0.9012857650552965, 'cs': 0.9012857650552965}


In [92]:
gower_dist_complete = gower_matrix(test_data_complete['X_complete'],cat_features=cat_mask)

print(clust_utils.internal_metrics(predicted_labels, gower_dist_complete, test_data_complete['X_complete']))
print(clust_utils.internal_metrics(predicted_mica, gower_dist_complete, test_data_complete['X_complete']))
print(clust_utils.internal_metrics(predicted_kpod[0], gower_dist_complete, test_data_complete['X_complete']))
print(clust_utils.internal_metrics(result, gower_dist_complete, test_data_complete['X_complete']))

{'sh': 0.30691099166870117, 'ch': 19.991120256855428, 'db': 4.569713701886209}
{'sh': 0.01969486102461815, 'ch': 64.46168206660893, 'db': 2.2811610033994887}
{'sh': -0.09899629652500153, 'ch': 36.8486694449497, 'db': 4.825612872744771}
{'sh': 0.16567306220531464, 'ch': 8.85324555127342, 'db': 5.502872371451179}
